In [1]:
import ast_parser
from ast_parser import ProjectParser, SymbolRecord

In [2]:
parser = ProjectParser("../example_repo/")
parser.build_index()
print(parser.get_all_files())

[WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/conf.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/toy-example.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample/setup.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample/eggsample/hookspecs.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample/eggsample/host.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample/eggsample/lib.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample/eggsample/__init__.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample-spam/eggsample_spam.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/docs/examples/eggsample-spam/setup.py'), WindowsPath('C:/Users/mm615/CS 329A/ReCAP-

In [3]:
for rec in parser.list_symbols("pluggy/testing/benchmark.py"):
    print(rec.kind, rec.qualname, rec.position.start_line)

function hook 21
function wrapper 26
function hooks 31
function wrappers 36
function test_hook_and_wrappers_speed 40
function setup 41
function test_call_hook 70
class HookSpec 73
method HookSpec.fun 75
class Plugin 78
method Plugin.__init__ 79
method Plugin.__repr__ 82
method Plugin.fun 86
class PluginWrap 90
method PluginWrap.__init__ 91
method PluginWrap.__repr__ 94
method PluginWrap.fun 98


In [4]:
# Get symbol record from qualname
rec = parser.resolve("pluggy/testing/benchmark.py", "PluginWrap")
print(rec)
print(rec.qualname)
print(rec.position.start_line, rec.position.end_line)

SymbolRecord(file=WindowsPath('C:/Users/mm615/CS 329A/ReCAP-SWE/example_repo/pluggy/testing/benchmark.py'), qualname='PluginWrap', kind='class', position=Position(start=(90, 4), end=(99, 26)))
PluginWrap
90 99


In [5]:
# print 源代码
print(parser.get_source(rec))             
print('=' * 40)
print(parser.code_with_lineno(parser.get_source(rec),
                            start_line=rec.position.start_line))
print('=' * 40)
print(parser.get_source_with_context(rec, before=10, after=10))

class PluginWrap:
        def __init__(self, num: int) -> None:
            self.num = num

        def __repr__(self) -> str:
            return f"<PluginWrap {self.num}>"

        @hookimpl(wrapper=True)
        def fun(self):
            return (yield)
  90 | class PluginWrap:
  91 |         def __init__(self, num: int) -> None:
  92 |             self.num = num
  93 | 
  94 |         def __repr__(self) -> str:
  95 |             return f"<PluginWrap {self.num}>"
  96 | 
  97 |         @hookimpl(wrapper=True)
  98 |         def fun(self):
  99 |             return (yield)
            self.num = num

        def __repr__(self) -> str:
            return f"<Plugin {self.num}>"

        @hookimpl
        def fun(self, hooks, nesting: int) -> None:
            if nesting:
                hooks.fun(hooks=hooks, nesting=nesting - 1)

    class PluginWrap:
        def __init__(self, num: int) -> None:
            self.num = num

        def __repr__(self) -> str:
            return f"<Plug

In [6]:
parser.edit_symbol(
    "pluggy/testing/benchmark.py",
    "PluginWrap.__init__",
    relative_line=2,
    new_code="print('Edited line inserted here')",
    mode="insert"
)

rec = parser.resolve("pluggy/testing/benchmark.py", "PluginWrap")
out =  parser.get_source(rec)
print( out )

class PluginWrap:
        def __init__(self, num: int) -> None:
            self.num = num
            print('Edited line inserted here')

        def __repr__(self) -> str:
            return f"<PluginWrap {self.num}>"

        @hookimpl(wrapper=True)
        def fun(self):
            return (yield)


In [7]:
for file in parser.get_all_files():
    for rec in parser.list_symbols(file):
       print(rec.qualname, rec.kind)

configure_logging function
WarnLogFilter class
WarnLogFilter.filter method
setup function
MySpec class
MySpec.myhook method
Plugin_1 class
Plugin_1.myhook method
Plugin_2 class
Plugin_2.myhook method
eggsample_add_ingredients function
eggsample_prep_condiments function
main function
get_plugin_manager function
EggsellentCook class
EggsellentCook.__init__ method
EggsellentCook.add_ingredients method
EggsellentCook.prepare_the_food method
EggsellentCook.serve_the_food method
eggsample_add_ingredients function
eggsample_prep_condiments function
eggsample_add_ingredients function
eggsample_prep_condiments function
create_branch function
get_upstream function
pre_release function
changelog function
main function
main function
run_old_style_hookwrapper function
_raise_wrapfail function
_warn_teardown_exception function
_multicall function
HookspecOpts class
HookimplOpts class
HookspecMarker class
HookspecMarker.__init__ method
HookspecMarker.__call__ method
HookspecMarker.setattr_hookspec_op

In [ ]:
from code_tree import CodeSemanticTree

tree = CodeSemanticTree(parser, project_name=parser.root.name)
tree.build_from_parser(parser)

In [9]:
tree.nodes['root']

CodeNode(id='root', name='example_repo', kind='project', parent=None, children=['C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\conf.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\toy-example.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample\\setup.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample\\eggsample\\hookspecs.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample\\eggsample\\host.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample\\eggsample\\lib.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample\\eggsample\\__init__.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample-spam\\eggsample_spam.py', 'C:\\Users\\mm615\\CS 329A\\ReCAP-SWE\\example_repo\\pluggy\\docs\\examples\\eggsample-spam\\

In [ ]:
tree.nodes['root'].children

In [10]:
def call_llm(prompt: str) -> str:
    return "This is a placeholder summary generated by LLM."

# 1) 用ProjectParser 建树
# parser = ProjectParser("../example_repo")
# parser.build_index()

tree = CodeSemanticTree(parser, project_name=parser.root.name)
tree.build_from_parser(parser)

# 为文件/类节点填充 LLM result
for nid, n in tree.nodes.items():
    n.summary = call_llm(f"Please summarize the following code for a code completion task: {n.kind} {n.name}: \n{...}")

# 2) 查看层次
tree.display()

# 3) 在 code completion 场景收集上下文（从 root 出发沿相关方向下钻）
context = tree.collect_context(
    query="insert text into buffer and update cursor",
    token_budget=1200,
    include_order=("ancestors", "self", "siblings", "children"),
    include_mode="best",  # 叶子优先源码，非叶子优先摘要
)

# 4) 把 context 拼到 prompt 里
prompt_parts = []
for nid, text in context.items():
    node = tree.get(nid)
    header = f"### {node.kind.upper()} {node.name}"
    prompt_parts.append(header + "\n" + text)
final_context_block = "\n\n".join(prompt_parts)
final_context_block

- project: example_repo  
  - file: conf.py  
    - function: configure_logging  
    - class: WarnLogFilter  
      - method: WarnLogFilter.filter  
    - function: setup  
  - file: toy-example.py  
    - class: MySpec  
      - method: MySpec.myhook  
    - class: Plugin_1  
      - method: Plugin_1.myhook  
    - class: Plugin_2  
      - method: Plugin_2.myhook  
  - file: setup.py  
  - file: hookspecs.py  
    - function: eggsample_add_ingredients  
    - function: eggsample_prep_condiments  
  - file: host.py  
    - function: main  
    - function: get_plugin_manager  
    - class: EggsellentCook  
      - method: EggsellentCook.__init__  
      - method: EggsellentCook.add_ingredients  
      - method: EggsellentCook.prepare_the_food  
      - method: EggsellentCook.serve_the_food  
  - file: lib.py  
    - function: eggsample_add_ingredients  
    - function: eggsample_prep_condiments  
  - file: __init__.py  
  - file: eggsample_spam.py  
    - function: eggsample_add_ingre

'### PROJECT example_repo\nThis is a placeholder summary generated by LLM.\n\n### FILE conf.py\nfrom importlib import metadata\nfrom typing import TYPE_CHECKING\n\n\nif TYPE_CHECKING:\n    import sphinx.application\n\n\nextensions = [\n    "sphinx.ext.autodoc",\n    "sphinx.ext.doctest",\n    "sphinx.ext.intersphinx",\n    "sphinx.ext.coverage",\n    "sphinx.ext.viewcode",\n]\n\n# Add any paths that contain templates here, relative to this directory.\ntemplates_path = ["_templates"]\n\nsource_suffix = ".rst"\n\n# The master toctree document.\nmaster_doc = "index"\n\n# General information about the project.\n\nproject = "pluggy"\ncopyright = "2016, Holger Krekel"\nauthor = "Holger Krekel"\n\nrelease = metadata.version(project)\n# The short X.Y version.\nversion = ".".join(release.split(".")[:2])\n\n\nlanguage = "en"\n\npygments_style = "sphinx"\n# html_logo = "_static/img/plug.png"\nhtml_theme = "alabaster"\nhtml_theme_options = {\n    "logo": "img/plug.png",\n    "description": "The py